### yahoofinance 크롤링

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

In [40]:
# 티커 배열
tickers = ['SOXL', 'VOO', 'SPY']  # 예시로 몇 가지 티커 추가

# Edge 옵션 설정 (헤드리스 모드: 브라우저 창을 띄우지 않음)
edge_options = webdriver.EdgeOptions()
edge_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
edge_options.add_argument("--no-sandbox")
edge_options.add_argument("--disable-dev-shm-usage")

# Edge 웹드라이버 설정
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=edge_options)

# 뉴스 크롤링 함수
def crawl_news_for_ticker(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/news/"
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기
    
    # 뉴스 제목 추출
    try:
        news_items = driver.find_elements(By.XPATH, '//h3[@class="clamp  yf-1sxfjua"]')  # 뉴스 제목이 들어있는 h3 태그
        print(f"Ticker: {ticker} - 뉴스 제목들:")
        for news in news_items:
            title = news.text
            print(f"- {title}")
    except Exception as e:
        print(f"Error retrieving news for {ticker}: {str(e)}")

# 각 티커에 대해 뉴스 제목 크롤링
for ticker in tickers:
    crawl_news_for_ticker(ticker)

# 브라우저 종료
driver.quit()


Ticker: SOXL - 뉴스 제목들:
- Investors Bought Up IWM, RSP Last Week
- Direxion META Single-Stock ETFs Wrap Mag-7 Lineup
- ETFs Pulled in $87B in May, Almost Triple April's Total
- ETF Spotlight: SOXL Seesaws as Nvidia Sags
- What the Kentucky Derby and ETFs Have in Common
- SOXL Sees 10% AUM Gain; ETF Weekly Flows
Ticker: VOO - 뉴스 제목들:
- Why Gen Z bets big while boomers play it safe: A generational breakdown of market returns
- ETFs Set Quarterly Inflows Record $280B in Q3
- Here's how AI is set to disrupt healthcare — albeit slowly
- Equal Weight ETFs Record Over $2B in September Inflows
- U.S. ETFs Hit $10T in Assets on Market Gains, Inflows
- Fed Rate Cut Signals Inflation Is Licked
- Major ETFs Climb After Fed Slashes Interest Rate
- U.S. ETF Industry Assets Climb to Record $9.7T in August
- VOO, IVV Gain Ground on SPY in S 500 ETF Race
- What investors are getting wrong about the VIX right now
- VOO, TLT Push August ETF Inflows to $71B
- Why investors turn to ETFs during times of mark